This is the version that can load weights previously trained with 40k X-ring scattering images (128 x 128) at ALS by on a Linux machine. Loading the weights on Mac may show abnormal results.

In [ ]:
from ddim import DiffusionModel

import matplotlib.pyplot as plt
import tensorflow as tf
#import tensorflow_datasets as tfds

from tensorflow import keras
from keras import layers

import numpy as np

In [ ]:
# need load new dataset because the model need the mean and var information
batch_size = 1
data_dir = './data/dataset'
train_dataset = tf.keras.utils.image_dataset_from_directory(data_dir,
                                                        labels=None, #if label='inferred', data_dir should be structured
                                                        color_mode='rgb', #'grayscale',
                                                        batch_size=batch_size,
                                                        image_size=(128, 128),
                                                        shuffle=True,
                                                        seed=1,
                                                    #  validation_split=0.2,
                                                    #  subset='both',
                                                        interpolation='lanczos5')    
    
normalization_layer = layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x: (normalization_layer(x))).prefetch(buffer_size=32) 

In [ ]:
model = DiffusionModel(128, [32, 64, 96, 128], 2)

# to-do: generate new images with full resolutions (this would probably improve with more data for the training).  
# calculate mean and variance of training dataset for normalization
model.normalizer.adapt(train_dataset)

# checkpoint training for 2000 epochs 
checkpoint_path = "checkpoints4/diffusion_model"

# load the best model (the best model is saved based on the kid_val in validation) and generate images
model.load_weights(checkpoint_path)

In [ ]:
# This will genenerate new X-ray scattering images 
model.plot_gray_images()

### Add noise to a given image and denoise

In [ ]:
# plot the input image
input_image = next(iter(train_dataset))[0]
plt.imshow(input_image)

In [ ]:
model.generate_images(num_images=1, n=0, input_images=input_image, noise_level=1.0, outname='output_image.png')